In [ ]:
!pip install transformers[torch]
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 32.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00


In [147]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [148]:
import pandas as pd

In [149]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [150]:
my_data = pd.read_csv("/content/train_toknized_df.csv",index_col=0)

In [151]:
my_data.shape

(3178, 3)

In [152]:
from ast import literal_eval
import transformers
my_data["label"]=my_data["label"].apply(lambda x :literal_eval(x))
my_data["sentence"]=my_data["sentence"].apply(lambda x :literal_eval(x))


In [153]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix-ner")
model = AutoModelForTokenClassification.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix-ner")
model = model.to('cuda')

In [154]:
def toknize_text(example):
  return tokenizer(example,truncation = True ,max_length =512)

In [155]:
len(my_data.label[0])

25

In [156]:
len(my_data.sentence[0])

25

In [157]:
tokenized_data = []
new_labels = []
for index , sentence in enumerate(my_data["sentence"]):
  tokenized_sentence = []
  label = my_data.iloc[index,2]
  for x, word in enumerate(sentence):
     if len(tokenizer.tokenize(word,truncation = True ,max_length =512)) > 1 :
       count =len(tokenizer.tokenize(word,truncation = True ,max_length =512))
       while count > 1:
         label.insert(x+1,my_data.iloc[index,2][x])
         count-=1
     tokenized_sentence.extend(tokenizer.tokenize(word,truncation = True ,max_length =512))
  new_labels.append(label)
  tokenized_data.append(tokenized_sentence)

In [158]:
my_dict = {"sentence":tokenized_data,"label":new_labels}

In [159]:
toknized_df_train = pd.DataFrame(my_dict)

In [160]:
my_data1 = pd.read_csv("/content/eval_toknized_df.csv",index_col=0)

In [161]:
my_data1["label"]=my_data1["label"].apply(lambda x :literal_eval(x))
my_data1["sentence"]=my_data1["sentence"].apply(lambda x :literal_eval(x))

In [162]:
tokenized_data = []
new_labels = []
for index , sentence in enumerate(my_data1["sentence"]):
  tokenized_sentence = []
  label = my_data1.iloc[index,2]
  for x, word in enumerate(sentence):
    if len(tokenizer.tokenize(word,truncation = True ,max_length =512)) > 1 :
      count =len(tokenizer.tokenize(word,truncation = True ,max_length =512))
      while count > 1:
        label.insert(x+1,my_data1.iloc[index,2][x])
        count-=1
    tokenized_sentence.extend(tokenizer.tokenize(word,truncation = True ,max_length =512))
  new_labels.append(label)
  tokenized_data.append(tokenized_sentence)

In [163]:
my_dict1 = {"sentence":tokenized_data,"label":new_labels}

In [164]:
toknized_df_eval = pd.DataFrame(my_dict1)

In [165]:
my_sentences_train  = []
for sen in toknized_df_train["sentence"]:
  my_sentences_train.append(sen)

In [166]:
my_sentences_eval  = []
for sen in toknized_df_eval["sentence"]:
  my_sentences_eval.append(sen)

In [167]:
my_label_train = []
for label in toknized_df_train["label"]:
  my_label_train.append(label)

In [168]:
my_label_eval = []
for label in toknized_df_eval["label"]:
  my_label_eval.append(label)

In [169]:
from datasets import ClassLabel

labels = ['B-LOC', 'O', 'B-ORG', 'I-ORG', 'B-PERS', 'I-PERS', 'I-LOC', 'B-MISC', 'I-MISC']
c2l = ClassLabel(num_classes=9, names=labels)

embeded_labels_train = [c2l.str2int(label) for label in my_label_train ]
embeded_labels_eval = [c2l.str2int(label) for label in my_label_eval ]

In [170]:
def pad_labels(lst):
  new_labels = []
  for item in lst:
    length_item = len(item)
    max_len = 512
    if length_item >512:
      del item[512:]
    diff = max_len - length_item
    item.extend([-100]*diff)
    new_labels.append(item)
  return new_labels

In [171]:
embeded_labels_train = pad_labels(embeded_labels_train)
embeded_labels_eval = pad_labels(embeded_labels_eval)

In [172]:
train_encodings = tokenizer(my_sentences_train,padding='max_length', truncation=True, max_length=512, is_split_into_words=True)
eval_encodings = tokenizer(my_sentences_eval,padding='max_length', truncation=True, max_length=512, is_split_into_words=True)

In [173]:
train_encodings.tokens()

['[CLS]',
 'وارتفع',
 'عدد',
 'العاط',
 '#',
 '#',
 'لين',
 'عن',
 'العمل',
 'الذين',
 'يحصل',
 '#',
 '#',
 'ون',
 'على',
 'إع',
 '#',
 '#',
 'انا',
 '##ت',
 'بط',
 '#',
 '#',
 'ا',
 '##لة',
 'بواقع',
 '2000',
 'في',
 'يوليو',
 '/',
 'تموز',
 'وهو',
 'عدد',
 'أقل',
 'من',
 'المتوقع',
 'لت',
 '#',
 '#',
 'ظل',
 'النسبة',
 'عند',
 '3',
 '%',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]'

In [174]:
len(train_encodings['input_ids'][0])

512

In [175]:
# Check the length of sentences VS labels for eval dataset

# for i,element in enumerate(my_sentences_eval):
#   if len(element) != len(embeded_labels_eval[i]):
#     print(len(eval_encodings['input_ids']),'||',len(embeded_labels_eval[i]))

In [176]:
train_encodings.input_ids[0]

[2,
 26914,
 2548,
 17696,
 7,
 7,
 5133,
 1955,
 2854,
 2677,
 6740,
 7,
 7,
 2263,
 1961,
 3820,
 7,
 7,
 2315,
 1005,
 2522,
 7,
 7,
 387,
 2035,
 17553,
 8352,
 1912,
 5457,
 19,
 10881,
 2400,
 2548,
 6794,
 1908,
 9617,
 2219,
 7,
 7,
 4812,
 17930,
 2192,
 23,
 9,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [177]:
print(train_encodings['input_ids'][0])

[2, 26914, 2548, 17696, 7, 7, 5133, 1955, 2854, 2677, 6740, 7, 7, 2263, 1961, 3820, 7, 7, 2315, 1005, 2522, 7, 7, 387, 2035, 17553, 8352, 1912, 5457, 19, 10881, 2400, 2548, 6794, 1908, 9617, 2219, 7, 7, 4812, 17930, 2192, 23, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [178]:
import torch

In [179]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [180]:
train_dataset = Dataset(train_encodings, embeded_labels_train)
val_dataset = Dataset(eval_encodings, embeded_labels_eval)

In [181]:
!pip install nervaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [182]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [183]:
# using Scikit learn

# def compute_metrics(p):
#     print(type(p))
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)
#     # print('::::::::Prediction::::::::')
#     # print(pred)
#     # print('::::::::TRUE LABELS::::::::')
#     # print(labels)

#     # accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     # recall = recall_score(y_true=labels, y_pred=pred)
#     # precision = precision_score(y_true=labels, y_pred=pred)
#     # f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [184]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Compute Metric

In [217]:
# from evaluate import load
# def compute_metrics(p):
#   pred, labels = p
#   metric = load('seqeval')
#   result = metric.compute(predictions=pred,refrences=labels)
#   print(result)

# from evaluate import evaluator

label_mapper = {0:'B-LOC', 1:'O', 2:'B-ORG', 3:'I-ORG', 4:'B-PERS', 5:'I-PERS', 6:'I-LOC', 7:'B-MISC', 8:'I-MISC'}
# def compute_metrics(p):
#   pred, labels = p
#   task_evaluator = evaluator("token-classification")
#   eval_results = task_evaluator.compute(
#     model_or_pipeline=model,
#     data=val_dataset,
#     metric="seqeval"
# )
def compute_metrics(p):
    predictions, labels = p
    print(predictions[0])
    #select predicted index with maximum logit for each token
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_mapper[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_mapper[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    # print(true_predictions[0])
    # print(true_labels[0])
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [186]:
# pip install accelerate -U

In [232]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    evaluation_strategy = 'epoch',
    max_steps = 398

)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [188]:
max = 0
for lst in embeded_labels_train:
  if len(lst)>max:
    max = len(lst)
max

512

In [233]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.333704,0.432773,0.321731,0.369081,0.905087


[[-1.4349065  11.276694   -2.0236952  ... -2.179596   -1.169476
  -2.3007588 ]
 [-0.38033998 11.050698   -2.3487236  ... -2.299264   -1.2520903
  -2.3171709 ]
 [-0.7900127  10.728137   -2.5843868  ... -2.1046276  -0.8697945
  -2.115697  ]
 ...
 [ 0.29776454 10.367735   -1.412705   ... -1.8737606  -0.5549633
  -2.6449351 ]
 [ 0.75923413  9.881911   -0.930629   ... -2.0909317  -0.7034209
  -2.7624211 ]
 [ 0.6484379   9.907923   -1.0313877  ... -1.902905   -0.8649752
  -2.655882  ]]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

TrainOutput(global_step=398, training_loss=0.13990851742538377, metrics={'train_runtime': 304.1377, 'train_samples_per_second': 10.469, 'train_steps_per_second': 1.309, 'total_flos': 830453646145536.0, 'train_loss': 0.13990851742538377, 'epoch': 1.0})

In [219]:
trainer.evaluate()

[[-0.439255    8.169909   -1.3448981  ... -2.3962238   0.11613086
  -0.77194786]
 [-0.35689867  8.074369   -1.6202753  ... -2.4128969   0.06231994
  -0.55470145]
 [ 0.26799828  7.11687    -1.4858155  ... -2.2554586  -0.45800996
  -0.8146334 ]
 ...
 [ 0.12458007  8.286712   -1.6015327  ... -1.9233792   0.28710657
  -1.050792  ]
 [ 0.09917319  8.291827   -1.6115056  ... -2.0044441   0.23290907
  -1.0133768 ]
 [ 0.09656636  8.291482   -1.654681   ... -1.9999509   0.24057376
  -0.9725971 ]]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

{'eval_loss': 0.3123859465122223,
 'eval_precision': 0.6130434782608696,
 'eval_recall': 0.18875502008032127,
 'eval_f1': 0.2886386898669396,
 'eval_accuracy': 0.9066889404063821,
 'eval_runtime': 25.8755,
 'eval_samples_per_second': 30.724,
 'eval_steps_per_second': 3.865}

In [137]:
trainer.predict(val_dataset)

PredictionOutput(predictions=array([[[-0.7407001 ,  9.812473  , -1.056224  , ..., -2.6984103 ,
         -0.88885665, -2.0724277 ],
        [-0.8063749 ,  9.510979  , -0.69606084, ..., -3.012723  ,
         -0.55777186, -1.6879226 ],
        [ 0.25332987,  8.929947  , -1.086809  , ..., -2.7870317 ,
         -0.2350462 , -1.8365046 ],
        ...,
        [12.147505  , -1.5325849 , -1.8848431 , ..., -1.9871733 ,
         -1.9043987 , -2.2217019 ],
        [12.188751  , -1.4888438 , -1.9092426 , ..., -2.0353937 ,
         -1.8824595 , -2.1935956 ],
        [12.156139  , -1.4424438 , -1.8578966 , ..., -2.0451238 ,
         -1.8550837 , -2.2254825 ]],

       [[-0.3621593 ,  9.85907   , -1.0331396 , ..., -2.9934807 ,
         -0.6198076 , -2.3851607 ],
        [-1.3439219 ,  9.727205  , -0.7453819 , ..., -3.0114157 ,
         -0.95295733, -1.7614315 ],
        [-1.1783129 ,  9.367467  , -0.260604  , ..., -2.7939034 ,
         -1.2359033 , -2.1787457 ],
        ...,
        [12.135889  , -1.

In [139]:
pred= trainer.predict(val_dataset)
labels = embeded_labels_eval
metric = load('seqeval')
result = metric.compute(predictions=pred,refrences=labels)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/evaluate/module.py:485 in add_batch                      │
│                                                                                                  │
│   482 │   │   try:                                                                               │
│   483 │   │   │   for key, column in batch.items():                                              │
│   484 │   │   │   │   if len(column) > 0:                                                        │
│ ❱ 485 │   │   │   │   │   self._enforce_nested_string_type(self.selected_feature_format[key],    │
│   486 │   │   │   batch = self.selected_feature_format.encode_batch(batch)                       │
│   487 │   │   │   self.writer.write_batch(batch)                                                 │
│   488 │   │   except (pa.ArrowInvalid, TypeError):                                               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/evaluate/module.py:723 in _enforce_nested_string_type    │
│                                                                                                  │
│   720 │   │   │   │   │   │   if _check_non_null_non_empty_recursive(first_elmt, schema.featur   │
│   721 │   │   │   │   │   │   │   break                                                          │
│   722 │   │   │   │   │   if not isinstance(first_elmt, list):                                   │
│ ❱ 723 │   │   │   │   │   │   return self._enforce_nested_string_type(schema.feature, first_el   │
│   724 │   │                                                                                      │
│   725 │   │   elif isinstance(schema, Value):                                                    │
│   726 │   │   │   if pa.types.is_string(schema.pa_type) and not isinstance(obj, str):            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/evaluate/module.py:727 in _enforce_nested_string_type    │
│                                                                                                  │
│   724 │   │                                                                                      │
│   725 │   │   elif isinstance(schema, Value):                                                    │
│   726 │   │   │   if pa.types.is_string(schema.pa_type) and not isinstance(obj, str):            │
│ ❱ 727 │   │   │   │   raise TypeError(f"Expected type str but got {type(obj)}.")                 │
│   728                                                                                            │
│   729                                                                                            │
│   730 class Metric(EvaluationModule):                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: Expected type str but got <class 'numpy.ndarray'>.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:4                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/evaluate/module.py:432 in compute                        │
│                                                                                                  │
│   429 │   │   compute_kwargs = {k: kwargs[k] for k in kwargs if k not in self._feature_names()   │
│   430 │   │                                                                                      │
│   431 │   │   if any(v is no

In [140]:
compute_metrics((trainer.predict(val_dataset),embeded_labels_eval))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in compute_metrics:14                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/evaluate/evaluator/token_classification.py:244 in        │
│ compute                                                                                          │
│                                                                                                  │
│   241 │   │   self.check_for_mismatch_in_device_setup(device, model_or_pipeline)                 │
│   242 │   │                                                                                      │
│   243 │   │   # Prepare inputs                                                                   │
│ ❱ 244 │   │   data = self.load_data(data=data, subset=subset, split=split)                       │
│   245 │   │   metric_inputs, pipe_inputs = self.prepare_data(                                    │
│   246 │   │   │   data=data, input_column=input_column, label_column=label_column, join_by=joi   │
│   247 │   │   )                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/evaluate/evaluator/base.py:352 in load_data              │
│                                                                                                  │
│   349 │   │   │   │   logger.warning("`data` is a preloaded Dataset! Ignoring `subset` and `sp   │
│   350 │   │   │   return data                                                                    │
│   351 │   │   else:                                                                              │
│ ❱ 352 │   │   │   raise ValueError(                                                              │
│   353 │   │   │   │   "Please specify a valid `data` object - either a `str` with a name or a    │
│   354 │   │   │   )                                                                              │
│   355                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Please specify a valid `data` object - either a `str` with a name or a `Dataset` object.